In [1]:
from itertools import product
import pandas as pd
from mip import BINARY, Model, maximize, xsum
from more_itertools import pairwise, windowed

In [2]:
shifts = ["日", "夜", "休"] # シフトリスト
dfws = pd.read_csv("wish.csv") # 希望シフト
days = dfws.columns[1:] # 日付リスト
dffx = dfws.melt("Name", days, "Day", "Shift").dropna() # モデルに使う希望リスト
print(dffx)

   Name Day Shift
0    佐藤  D1     休
5    田中  D2     休
10   鈴木  D3     休
31   高橋  D8     休


In [3]:
d = product(dfws.Name, days, shifts)
df = pd.DataFrame(d, columns=dffx.columns)
print(df)

   Name Day Shift
0    佐藤  D1     日
1    佐藤  D1     夜
2    佐藤  D1     休
3    佐藤  D2     日
4    佐藤  D2     夜
..  ...  ..   ...
91   高橋  D7     夜
92   高橋  D7     休
93   高橋  D8     日
94   高橋  D8     夜
95   高橋  D8     休

[96 rows x 3 columns]


In [4]:
m = Model()

In [5]:
x = m.add_var_tensor((len(df),), "x", var_type=BINARY)

In [6]:
df["Var"] = x
df

,Name,Day,Shift,Var
0,佐藤,D1,日,x_0
1,佐藤,D1,夜,x_1
2,佐藤,D1,休,x_2
3,佐藤,D2,日,x_3
4,佐藤,D2,夜,x_4
...,...,...,...,...
91,高橋,D7,夜,x_91
92,高橋,D7,休,x_92
93,高橋,D8,日,x_93
94,高橋,D8,夜,x_94


In [7]:
m.objective = maximize(xsum(dffx.merge(df).Var))

### 制約条件

In [8]:
for _, gr in df.groupby(["Name", "Day"]):
    m += xsum(gr.Var) == 1 # 看護師と日付の組み合わせごとにシフトは1つ
for _, gr in df.groupby("Day"):
    m += xsum(gr[gr.Shift == "日"].Var) >= 2 # 日付ごとに日勤は2以上
    m += xsum(gr[gr.Shift == "夜"].Var) >= 1 # 日付ごとに夜勤は1以上

In [18]:
q1 = '(Day == @d1 & Shift == "夜")|'
q2 = '(Day == @d1 & Shift != "休")'
q3 = 'Day in @dd & Shift == "休"'

In [20]:
for _, gr in df.groupby("Name"):
    m += xsum(gr[gr.Shift == "日"].Var) <= 4 # 看護婦ごとに日勤は4以上
    m += xsum(gr[gr.Shift == "夜"].Var) <= 2 # 看護婦ごとに夜勤は2以上
    for d1, d2 in pairwise(days):
        m += xsum(gr.query(q1 + q2, engine='python').Var) <= 1 # 夜勤と翌日休みはどちらかだけ
    for dd in windowed(days, 4):
        m += xsum(gr.query(q3, engine='python').Var) >= 1 # 4連続勤務のうち休みは1日以上

more_itertools.pairwise（反復可能）[ソース]  
元のアイテムから、重複するペアアイテムのイテレータを返します  

take(4, pairwise(count()))  
[(0, 1), (1, 2), (2, 3), (3, 4)]  

In [21]:
print(days)

Index(['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8'], dtype='object')


In [22]:
for d1, d2 in pairwise(days):
    print(d1, d2)

D1 D2
D2 D3
D3 D4
D4 D5
D5 D6
D6 D7
D7 D8


ore_itertools.windowed（seq、n、fillvalue = None、step = 1 ）[ソース]  
指定されたイテラブル上で幅nのスライディングウィンドウを返します。  

all_windows = windowed([1, 2, 3, 4, 5], 3)  
list(all_windows)  
[(1, 2, 3), (2, 3, 4), (3, 4, 5)]  

In [23]:
for dd in windowed(days, 4):
    print(dd)

('D1', 'D2', 'D3', 'D4')
('D2', 'D3', 'D4', 'D5')
('D3', 'D4', 'D5', 'D6')
('D4', 'D5', 'D6', 'D7')
('D5', 'D6', 'D7', 'D8')


In [24]:
m.optimize()

<OptimizationStatus.OPTIMAL: 0>

### 結果の作成

In [25]:
df["Val"] = df.Var.astype(float)

In [27]:
res = df[df.Val > 0]
res.head()

,Name,Day,Shift,Var,Val
2,佐藤,D1,休,x_2,1.0
3,佐藤,D2,日,x_3,1.0
6,佐藤,D3,日,x_6,1.0
9,佐藤,D4,日,x_9,1.0
14,佐藤,D5,休,x_14,1.0


In [28]:
res = res.pivot_table("Shift", "Name", "Day", "first")
res

Day,D1,D2,D3,D4,D5,D6,D7,D8
Name,,,,,,,,
佐藤,休,日,日,日,休,日,夜,夜
田中,日,休,日,夜,夜,休,日,日
鈴木,日,夜,休,日,日,夜,休,日
高橋,夜,日,夜,休,日,日,日,休


In [29]:
print(f"ステータス：{m.status}")
print(f"希望をかなえた数：{m.objective.x}")
print(res)

ステータス：OptimizationStatus.OPTIMAL
希望をかなえた数：4.0
Day  D1 D2 D3 D4 D5 D6 D7 D8
Name                        
佐藤    休  日  日  日  休  日  夜  夜
田中    日  休  日  夜  夜  休  日  日
鈴木    日  夜  休  日  日  夜  休  日
高橋    夜  日  夜  休  日  日  日  休
